# Advanced optimizers in Fed-BioMed


**Difficulty level**: **advanced**
    
## Introduction

This tutorial presents on how  to deal with heterogeneous dataset by changing its `Optimizer`. 
In `Fed-BioMed`, one can specify two sort of `Optimizer`s:

1. a `Optimizer` on the `Node` side, defined on the `Training Plan`
2. a `Optimizer` on the `Researcher` side, configured in the `Experiment`

Advanced `Optimizer` are backed by [`declearn` package](), a python package focused on `Optimization` for Federated Learning. Advanced `Optimizer` can be used regardless of the machine learning framework (compatible with both sklearn and PyTorch)


In this tutorial you will learn:
- how to use and chain one or several `Optimizers` on `Node` and `Researcher` side
- how to use fedopt
- how to use `Optimizers` that exchange auxiliary variables such as `Scaffold`

For further details you can refer to the [`Optimizer` section in the User Guide]()

# 1. Configuring `Nodes`

Before starting, we need to configure several `Nodes` and add MedNist dataset to it. Node configuration steps require `fedbiomed-node` conda environment. Please make sure that you have the necessary conda environment: this is explained in the [installation tutorial](../../installation/0-basic-software-installation). 


Please open a terminal, `cd` to the base directory of the cloned fedbiomed project and follow the steps below.    

* **Configuration Steps:**
    * Run `${FEDBIOMED_DIR}/scripts/fedbiomed_run node add` in the terminal
    * It will ask you to select the data type that you want to add. The third option has been configured to add the MedNIST dataset. Please type `3` and continue. 
    * Please use default tags which are `#MNIST` and `#dataset`.
    * For the next step, please select the directory that you want to download the MNIST dataset.
    * After the download is completed you will see the details of the MNIST dataset on the screen.
 
Please run the command below in the same terminal to make sure the MNIST dataset is successfully added to the Node. 

```
$ ${FEDBIOMED_DIR}/scripts/fedbiomed_run node config conf1.ini add
```

Before starting the node, please make sure that you have already launched the network using command `scripts/fedbiomed_run network`. Afterward, all you need to do is to start the node.


```
$ ${FEDBIOMED_DIR}/scripts/fedbiomed_run node config conf1.ini start
```

In another terminal, you may proceed by launching a second `Node`

# 2. Defining an `Optimizer` on `Node` side

`Optimizers` are defined through the `init_optimizer` method of the `training plan`. They must be set using `Fed-BioMed` `Optimizer` object (ie from `fedbiomed.common.optimizers.optimizer.Optimizer`)

## 2.1 With PyTorch framework

In [this tutorial]() we have showcased the use of a PyTorch model with [PyTorch native optimizers](), such as `torch.optim.SGD`. In the present tutorial, we will see how to use `declearn` cross frameworks optimizers


### PyTorch Training Plan
Below is a simple implementation of a `declearn` SGD `Optimizer`

In [ ]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms
from torchvision.models import densenet121
from fedbiomed.common.optimizers.optimizer import Optimizer

# Here we define the model to be used. 
# we will use the densnet121 model
class MyTrainingPlan(TorchTrainingPlan):
    
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms",
                "from torchvision.models import densenet121"]

        return deps
    
    def init_model(self):
        self.loss_function = torch.nn.CrossEntropyLoss()
        model = densenet121(pretrained=True)
        model.classifier =nn.Sequential(nn.Linear(1024,512), nn.Softmax())
        return model 
    
    def init_optimizer(self, optimizer_args):
        # Defines and return a declearn optimizer
        return Optimizer(lr=optimizer_args['lr'])

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MedNIST data
        print("dataset path",self.dataset_path)
        preprocess = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(
                                            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                                        )])
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = self.loss_function(output, target)
        return loss


### Sklearn `Training Plan`

# 3. Defining an `Optimizer` on `Researcher` side: `FedOpt`

# 4. Defining `Scaffold` through `Optimizer`

# 5. Explore `declearn` and the Fed-BioMed user guide